In [ ]:
import pandas as pd
import numpy as np
import dash
import flask
import math
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go
import dash_table
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__)
app.scripts.config.serve_locally = True

app.layout=html.Div(children=[
            html.H1(id='header',children=['Linear Regression Demonstration'],style={
                              'textAlign': 'center',
                              'color': 'blue',
                              'fontSize':50}),
            html.Div(id='instructions',
                     children=['Enter coordinates in table below and watch as Linear Regression model find the line of best fit',
                                                html.Br(),'You can select different metrics to see how the model performs'],
                    style={'textAlign': 'center',
                              'color': 'skyblue',
                              'fontSize':20}              
                    ),
            dcc.Graph(id='adding-rows-graph'),
            dcc.RadioItems(id='residuals',options=[{'label':'Residuals Off','value':'no'},
                                                   {'label':'Residuals On','value':'yes'}],value='no'),
            dcc.RadioItems(id='mean',options=[{'label':'Mean Off','value':'no'},
                                              {'label':'Mean On','value':'yes'}],value='no'),
            html.Br(),
            html.Div(id='metrics'),
            html.Br(),
            html.Div(id='input',children=[
                
            
                dash_table.DataTable(id='table', columns=[{'name':'x','id':'x','type':'numeric'},
                                                      {'name':'y','id':'y','type':'numeric'}],
                                             data=[{'x':5,'y':6}],
                                             editable=True,
                                             row_deletable=True),
                html.Button('Add Row', id='editing-rows-button', n_clicks=0)
                    ])
            
])   

@app.callback(
    Output('table', 'data'),
    [Input('editing-rows-button', 'n_clicks')],
    [State('table', 'data'),
     State('table', 'columns')])
def add_row(n_clicks, rows, columns):
    if n_clicks > 0:
        rows.append({c['id']: '' for c in columns})
    return rows 
@app.callback(
    [Output('adding-rows-graph', 'figure'),
     Output('metrics','children')],
    [Input('table', 'data'),
     Input('table', 'columns'),
     Input('residuals','value'),
     Input('mean','value')])
def display_output(rows, columns,resid,mean):
    df = pd.DataFrame(rows, columns=[c['name'] for c in columns])
    df['rd']=df.x.apply(lambda x: int(round(float(x),0)))
    x=[i for i in range(df.rd.max()+2)]
    mod=LinearRegression().fit(df[['x']],df[['y']])
    df['pred']=mod.predict(df[['x']])
    r2=mod.score(df[['x']],df[['y']])
    mae=mean_absolute_error(df[['y']],df[['pred']])
    rmse=mean_squared_error(df[['y']],df[['pred']])**0.5
    if mean=='yes':
        mn=df.y.mean()
    else:
        mn=[]
    children=['R^2 Score: ' + str(r2),html.Br(),'Mean Absolute Error: ' +str(mae),html.Br(),'Root Mean Squared Error: '+str(rmse)]
    fig=go.Figure()
    fig.add_trace(go.Scatter(
        x=[i for i in df['x']],
        y=[j for j in df['y']],
        mode='markers',
        showlegend=False
    )),
    fig.add_trace(go.Scatter(
        name='Regression Line',
        x=[i for i in x],
        y=[mod.intercept_[0]+mod.coef_[0][0]*x[i] for i in x],
       
    )),
    fig.add_trace(go.Scatter(
        name='Mean Line',
        x=[i for i in x],
        y=[mn for i in range(df.rd.max()+2)],
       
    )),
    if resid=='yes':
        for index in range(len(df.x)):
            if df.y[index]>df.pred[index]:
                fig.add_shape(
                    dict(type='line',x0=df.x[index],y0=df.pred[index],x1=df.x[index],y1=df.y[index]))
            else:
                fig.add_shape(
                    dict(type='line',x0=df.x[index],y0=df.y[index],x1=df.x[index],y1=df.pred[index]))   
      
    return fig,children
                                   
                                   
            

application=app.server
if __name__ == '__main__':
    
    application.run(debug=False)

<ipython-input-1-eae8aa43f9b4>:6: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-1-eae8aa43f9b4>:7: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-1-eae8aa43f9b4>:10: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Apr/2023 15:46:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2023 15:46:56] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_0_0m1638182159.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2023 15:46:56] "GET /_dash-component-suites/dash/deps/react@16.v2_0_0m1638182159.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2023 15:46:56] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_0_0m1638182159.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2023 15:46:56] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_0_0m1638182159.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2023 15:46:56] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_0_0m1638182159.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2023 15:46:56] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_0_0m1638182159.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2023 15:46:56] "GET /_dash-component-suites/d

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Nick\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Nick\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Nick\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Nick\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Nick\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Nick\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Nick\anaconda3\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_data(fun

127.0.0.1 - - [17/Apr/2023 15:47:23] "POST /_dash-update-component HTTP/1.1" 500 -


...

127.0.0.1 - - [17/Apr/2023 15:47:42] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Nick\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Nick\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Nick\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Nick\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Nick\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Nick\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Nick\anaconda3\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_data(fun

127.0.0.1 - - [17/Apr/2023 15:47:44] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Nick\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Nick\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Nick\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Nick\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Nick\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Nick\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Nick\anaconda3\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_data(fun

127.0.0.1 - - [17/Apr/2023 15:47:45] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [17/Apr/2023 15:47:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2023 15:47:53] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2023 15:47:53] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2023 15:47:53] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2023 15:47:53] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2023 15:47:53] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2023 15:47:53] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2023 15:47:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2023 15:47:53] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 200 -
C:\Users\Nick\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:682: UndefinedMetricWarnin